In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
import sys, os

print(f"Installing packages into environment {sys.executable}")

In [ ]:
!{sys.executable} -m pip install llama-index openai langchain azure-identity

# Get Auth Token

In [ ]:
from azure.identity import DefaultAzureCredential

In [ ]:
# Request credential
default_credential = DefaultAzureCredential()
token = default_credential.get_token("https://cognitiveservices.azure.com/.default")

# Configure Parameters

In [ ]:
aoai_base = "https://<aoai instance name>.openai.azure.com/"

# Setup Contexts

In [ ]:
from llama_index import StorageContext

In [ ]:
storage_context = StorageContext.from_defaults(persist_dir="./doc_store") # No persist_dir means all stores will be new Simple stores

In [ ]:
len(storage_context.docstore.docs)

## Setup custom LLM

In [ ]:
azure_kwargs={"api_type": "azure_ad", "api_version": "2023-03-15-preview", "api_base": aoai_base}

In [ ]:
from langchain.llms import AzureOpenAI
llm = AzureOpenAI(temperature=0.9, deployment_name="text-davinci-003", model_name="text-davinci-003", openai_api_key=token.token, model_kwargs=azure_kwargs)

In [ ]:
from llama_index import LLMPredictor

# define LLM
llm_predictor = LLMPredictor(llm)

In [ ]:
llm_predictor.get_llm_metadata()

## Setup custom Embedding

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from llama_index import LangchainEmbedding

In [ ]:
os.environ["OPENAI_API_KEY"]=token.token

In [ ]:
# load in AOAI embedding model from langchain
oai_embeddings = OpenAIEmbeddings(model="text-embedding-ada-002",
                              deployment="text-embedding-ada-002",
                              openai_api_key=token.token,
                              openai_api_base=azure_kwargs["api_base"],
                              openai_api_type=azure_kwargs["api_type"],
                              openai_api_version=azure_kwargs["api_version"],
                              chunk_size=1)

In [ ]:
embeddings = LangchainEmbedding(oai_embeddings)

In [ ]:
from llama_index import ServiceContext
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=embeddings) # using default chunk limit

# Create Index

In [ ]:
from llama_index import load_index_from_storage

In [ ]:
index = load_index_from_storage(storage_context=storage_context, service_context=service_context)

In [ ]:
# query will use the same embed_model
query_engine = index.as_query_engine(
    verbose=True, 
)


In [ ]:
%%time
response = query_engine.query(
    "How have emissions metrics reported in 2021 and 2022 differed?")
print(response)